In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

from utils import *
from usad import *

In [2]:
file = "MQTT_station1"
dataset = pd.read_csv(f'data/{file}.csv', nrows=30000)

dataset.head(2)

,Time,Message_Length,GPR_Topic,Area_Topic,VAR_Topic_1,VAR_Topic_2,Message_ID,Message_Value,Message_q
0,2023-09-05 16:39:46.584310,79,N143,T1,400,Ia,MF7,30.3,1
1,2023-09-05 16:39:47.536947,83,N143,LV1,GRP5,Ib,MF43,-28.3,1


In [3]:
dataset["label"] = 0
dataset["Time"] = pd.to_datetime(dataset["Time"])
dataset["Time"] = dataset["Time"].diff()
dataset["Time"] = pd.to_numeric(dataset["Time"])/1e6 #time in milliseconds
dataset = dataset.drop(index=0)

In [4]:
dataset.head(5)

,Time,Message_Length,GPR_Topic,Area_Topic,VAR_Topic_1,VAR_Topic_2,Message_ID,Message_Value,Message_q,label
1,952.637,83,N143,LV1,GRP5,Ib,MF43,-28.3,1,0
2,3209.656,78,N143,T1,400,Q,MF3,25.3,1,0
3,50.219,78,N143,T1,400,S,MF4,28.2,1,0
4,50.372,78,N143,T1,400,Q,MF3,26.4,1,0
5,2.017,79,N143,T1,400,Ia,MF7,35.4,1,0


In [6]:
# Transform all columns into float64
col = lambda column : [ord(char) for char in column]
for i in list(dataset):
    dataset[i]=dataset[i].apply(lambda column: ''.join(map(str, col(str(column))))  if not str(column).replace(".", "").isdigit() else str(column).replace("," , "."))
normal = dataset.astype(float)

In [7]:
normal.head(5)

,Time,Message_Length,GPR_Topic,Area_Topic,VAR_Topic_1,VAR_Topic_2,Message_ID,Message_Value,Message_q,label
1,952.637,83.0,78495251.0,768649.0,71828053.0,7398.0,77705251.0,4.550565e+09,1.0,0.0
2,3209.656,78.0,78495251.0,8449.0,400.0,81.0,777051.0,2.530000e+01,1.0,0.0
3,50.219,78.0,78495251.0,8449.0,400.0,83.0,777052.0,2.820000e+01,1.0,0.0
4,50.372,78.0,78495251.0,8449.0,400.0,81.0,777051.0,2.640000e+01,1.0,0.0
5,2.017,79.0,78495251.0,8449.0,400.0,7397.0,777055.0,3.540000e+01,1.0,0.0


In [8]:
temp = normal.drop(["label"], axis=1)
temp.to_csv(f'data/{file}.csv', header=False, index=False)